## Merge toute les données intérim (variables des 4 piliers)

In [22]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

external_data = Path('../data/external/')
processed_data = Path('../data/processed/')
raw_data = Path('../data/raw/')
interim_data = Path('../data/interim/')

In [27]:
!ls -lh ../data/interim/

total 9,0M
-rw-rw-r-- 1 outini outini 822K nov.  28 00:23 acces_point_phys_service_publique.csv
-rw-rw-r-- 1 outini outini  23K nov.  27 01:00 data_merge_V2.ipynb
-rw-rw-r-- 1 outini outini 4,0M nov.  28 00:23 data_scol_menage_demandeurEmplois.csv
-rw-rw-r-- 1 outini outini  79K nov.  27 01:00 IFN_var_scol_menage_demandeEmplois.ipynb
-rw-rw-r-- 1 outini outini 837K nov.  28 00:23 taux_couverture_mobile.csv
-rw-rw-r-- 1 outini outini 825K nov.  28 00:23 taux_couverture_THD.csv
-rw-rw-r-- 1 outini outini 706K nov.  28 00:23 taux_demandeurs_emploi.csv
-rw-rw-r-- 1 outini outini 781K nov.  28 00:23 taux_pauvrete.csv
-rw-rw-r-- 1 outini outini 999K nov.  28 00:23 variable_minimas_V2.csv


In [32]:
df1 = pd.read_csv(interim_data/'acces_point_phys_service_publique.csv')
df2 = pd.read_csv(interim_data/'data_scol_menage_demandeurEmplois.csv')
df3 = pd.read_csv(interim_data/'taux_couverture_mobile.csv')
df4 = pd.read_csv(interim_data/'taux_couverture_THD.csv')
df5 = pd.read_csv(interim_data/'taux_demandeurs_emploi.csv')    # get TX_DEMANDEUR_EMPLOIS from this one
df6 = pd.read_csv(interim_data/'taux_pauvrete.csv')
df7 = pd.read_csv(interim_data/'variable_minimas_V2.csv')

communes = pd.read_csv(raw_data/'table_insee_libcom_dep.csv').drop(columns=['Unnamed: 0'])

### Check pour avoir meme base de 3 index : (CODE_INSEE, LIBCOM, DEP) et meme taille de dataset

In [33]:
for data in [df1, df2, df3, df4, df5, df6, df7]:
    print(data.columns)
    print(data.shape)
    print()

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'ACCES_SERVICE_PUBLIC'], dtype='object')
(35013, 4)

Index(['Unnamed: 0', 'CODE_INSEE', 'LIBCOM', 'DEP', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS', 'TX_DEMANDEURS_EMPLOI'], dtype='object')
(36101, 10)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'COUVERTURE_MOBILE'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'DEP', 'LIBCOM', 'TAUX_COUVERTURE_THD'], dtype='object')
(35359, 4)

Index(['CODE_INSEE', 'LIBCOM', 'TX_DEMANDEUR_EMPLOIS'], dtype='object')
(35010, 3)

Index(['CODE_INSEE', 'LIBCOM', 'TX_POVERTY'], dtype='object')
(35010, 3)

Index(['Unnamed: 0', 'CODE_INSEE', 'LIBCOM', 'DEP', 'TX_RSA'], dtype='object')
(35010, 5)



### Check nouvelle taille de chaque dataset

In [34]:
indexes = ['CODE_INSEE', 'LIBCOM', 'DEP']
how = 'left'

df1 = communes.merge(df1, how=how, on=indexes).drop_duplicates(subset=indexes)
df2.drop(columns=['Unnamed: 0', 'TX_DEMANDEURS_EMPLOI'], inplace=True)
df2 = communes.merge(df2, how=how, on=indexes).drop_duplicates(subset=indexes)
df3 = communes.merge(df3, how=how, on=indexes).drop_duplicates(subset=indexes)
df4 = communes.merge(df4, how=how, on=indexes).drop_duplicates(subset=indexes)
df5 = communes.merge(df5, how=how, on=['CODE_INSEE', 'LIBCOM']).drop_duplicates(subset=indexes)
df6 = communes.merge(df6, how=how, on=['CODE_INSEE', 'LIBCOM']).drop_duplicates(subset=indexes)
df7.drop(columns=['Unnamed: 0'], inplace=True)

for data in [df1, df2, df3, df4, df5, df6, df7]:
    print(data.columns)
    print(data.shape)
    print()

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'ACCES_SERVICE_PUBLIC'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS'], dtype='object')
(35010, 8)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'COUVERTURE_MOBILE'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TAUX_COUVERTURE_THD'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_DEMANDEUR_EMPLOIS'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_POVERTY'], dtype='object')
(35010, 4)

Index(['CODE_INSEE', 'LIBCOM', 'DEP', 'TX_RSA'], dtype='object')
(35010, 4)



### MERGE

In [35]:
df = df1.merge(df2, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df3, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df4, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df5, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df6, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])
df = df.merge(df7, how='inner', on=['CODE_INSEE', 'LIBCOM', 'DEP'])

print(df.shape)

(35010, 14)


In [36]:
df

,CODE_INSEE,LIBCOM,DEP,ACCES_SERVICE_PUBLIC,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,COUVERTURE_MOBILE,TAUX_COUVERTURE_THD,TX_DEMANDEUR_EMPLOIS,TX_POVERTY,TX_RSA
0,01001,L'Abergement-Clémenciat,01,0,52.154281,22.823614,4.831504,23.779236,11.309305,NaN,1.0,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,01,0,40.425532,28.571429,4.761905,23.404255,7.978723,1.00,1.0,NaN,NaN,NaN
2,01004,Ambérieu-en-Bugey,01,0,49.474587,41.515231,10.216084,20.733915,16.620013,1.00,1.0,21.754928,26.170897,2.593516
3,01005,Ambérieux-en-Dombes,01,1,53.574678,21.076206,6.601983,19.488442,13.854125,1.00,1.0,NaN,NaN,NaN
4,01006,Ambléon,01,0,53.061224,36.363636,9.090909,26.530612,10.204082,0.65,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35005,97613,M'Tsangamouji,976,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
35006,97614,Ouangani,976,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
35007,97615,Pamandzi,976,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,51.411632,NaN
35008,97616,Sada,976,1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,49.198658,NaN


### Cela résulte en 11 colonnes pour calculer le score à savoir: 

In [38]:
list(df.drop(columns=indexes).columns)

['ACCES_SERVICE_PUBLIC',
 'TX_NSCOL15P',
 'TX_MENSEUL',
 'TX_FAMMONO',
 'TX_65ETPLUS',
 'TX_25ETMOINS',
 'COUVERTURE_MOBILE',
 'TAUX_COUVERTURE_THD',
 'TX_DEMANDEUR_EMPLOIS',
 'TX_POVERTY',
 'TX_RSA']

### + 3 colonnes d'index :

In [39]:
list(indexes)

['CODE_INSEE', 'LIBCOM', 'DEP']

In [40]:
df.to_csv(processed_data/'MERGE_data.csv', index=False)